# Gender classification

Sarah Nam initially wrote this notebook. Jae Yeon Kim reviwed the notebook, edited the markdown, and reproduced, commented on and made substantial changes in the code.

## Import libraries 

In [1]:

import pandas as pd
import numpy as np
import string

import collections
from collections import Counter
import matplotlib.pyplot as plt
import re

# NLTK
import nltk
import nltk as nlp
# nltk.download('punkt') You may need to download the dataset
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords

# ML

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB # Naive-Bayes
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression # Linear models
from xgboost import XGBClassifier # Xgboost

################### Validation ######################
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, LeavePOut, ShuffleSplit, StratifiedKFold

################### Vectorizer ######################
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import PCA

################### Model evals #####################
from sklearn.metrics import accuracy_score, balanced_accuracy_score, cohen_kappa_score, precision_score, recall_score

################### Imbalanced data #####################
from sklearn.utils import resample # for resampling

# Custom functions
from clean_text import clean_tweet


Bad key "text.kerning_factor" on line 4 in
/home/jae/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Load data

In [2]:

tweets = pd.read_csv('/home/jae/intersectional-bias-in-ml/raw_data/hatespeech_text_label_vote_RESTRICTED_100K.csv', sep='\t', header=None)

tweets.columns

Int64Index([0, 1, 2], dtype='int64')

In [3]:
# Name columns 
tweets.columns = ['text', 'label', 'votes']

# See dimentions 
tweets.shape

(99996, 3)

In [4]:
# dropping duplicates to remove the effects of boosting
#tweets = tweets.drop_duplicates()
#tweets.shape

## Clean text 

Borrowed the text clean code (`clean_text.py`) from racial classification to make the preprocessing step consistent across different classifiers.

In [5]:

# Clean text
tweets_clean = tweets.copy()

tweets_clean['text'] = clean_tweet(tweets_clean['text'])

tweets_clean.head()

,text,label,votes
0,beats by dr dre urbeats wired inear headphones...,spam,4
1,man it would fucking rule if we had a party ...,abusive,4
2,it is time to draw close to him 128591127995 f...,normal,4
3,if you notice me start to act different or dis...,normal,5
4,forget unfollowers i believe in growing 7 new ...,normal,3


## Import and wrangle training data

1. Sarah Nam found an open source Twitter data which was gender labeled (as in, it was written by a person of a certain gender):https://www.kaggle.com/crowdflower/twitter-user-gender-classification
2. The dataset includes image, username and other data. She only used the gender label and the sample tweet for training. 
3. The confidence level of the gender label was also used to sort out which data points would be useful for training our model. I decided that tweets with confidence level greater than .8 were to be used for training. This was because setting confidence threshold to 1 proved to return too few data points for training.

* Unfortunately, no clear explanation on what confidence means is provided in the Kaggle webpage. 

In [6]:
labeled_dat = pd.read_csv("/home/jae/intersectional-bias-in-ml/raw_data/gender_classified.csv", sep=",", engine='python')

training_data = labeled_dat[labeled_dat['gender:confidence'] > .8][['gender', 'text', 'gender:confidence']]

training_data['gender'].value_counts()

female     5371
male       4658
brand      3788
unknown     122
Name: gender, dtype: int64

In [7]:
# Getting rid of the unknown labeled data
training_data = training_data[training_data['gender'] != 'unknown']

# Clean text
training_data['text'] = clean_tweet(training_data['text']) 

training_data['text'].isnull().values.any()

False

- This dataset included not just male and female tweets, but also tweets by brand twitter accounts and some which were unknown. 
- Sarah Nam removed tweets for which the gender was unknown, and decided to use two dummy variables to encode the gender. One dummy variable was 1 for male and 0 for non-male (i.e. female or brand). A similar dummy variable was used for females. 
- Note that the classes are imbalanced in terms of their size.

In [8]:
training_data = training_data.copy()[['text', 'gender']]

# Inspect unique values 
training_data['gender'].unique()

array(['male', 'female', 'brand'], dtype=object)

In [9]:
# Create male and female columns 
training_data['male'] = [1 if i == 'male' else 0 for i in training_data['gender'].values]
training_data['female'] = [1 if i == 'female' else 0 for i in training_data['gender'].values]

In [10]:

# Check the class balance 

## Male
training_data['male'].value_counts()

0    9159
1    4658
Name: male, dtype: int64

In [11]:
## Female
training_data['female'].value_counts()

0    8446
1    5371
Name: female, dtype: int64

## Upsampling 

To fix the imbalance problem, Jae Kim randomly oversampled the minority class.

In [12]:
# Custom function for upsample. Jae Kim adapted some code from here: https://elitedatascience.com/imbalanced-classes 

def upsample(data, condition): 

    df_majority = data[data[condition] == 0]
    df_minority = data[data[condition] == 1]
    
    # Upsample (oversample) minority class 
    
    df_minority_upsampled = resample(df_minority, 
                                 replace = True,     # sample with replacement
                                 n_samples = 9000,    # to match majority class
                                 random_state = 1234) # reproducible results
    
    # Combine majority class with upsampled minority class
    data = pd.concat([df_majority, df_minority_upsampled])
    
    return(data)


## Feature extraction (bag-of-words model)

In [13]:
st = LancasterStemmer()

def token(text):
    txt = nlp.word_tokenize(text.lower())
    return [st.stem(word) for word in txt]


# Vectorizer

vectorizer = CountVectorizer(max_features = 4000, # 4,000 is large enough
                             min_df = 1, # minimum frequency 1
                             ngram_range = (1,2), # ngram 
                             tokenizer = token,
                             analyzer=u'word')

In [14]:
# Turn text into document-term matrix

def dtm_train(data, condition):
    
    ############################### Upsampling ################################

    data = upsample(data, condition)
    
    ############################### DOCUMENT-TERM MATRIX ################################
    
    # BOW model 
    
    features = vectorizer.fit_transform(data['text']).todense()
    
    # Response variable
    
    response = data[condition].values # values 

    ############################### STRATIFIED RANDOM SAMPLING ################################
    
    # Split into training and testing sets 

    X_train, X_test, y_train, y_test = train_test_split(features, response, 
                                                        test_size = 0.2, # training = 80%, test = 20%
                                                        random_state = 1234) 
    
    return(X_train, y_train, X_test, y_test)

In [15]:
# Male DTM

male_dtm = dtm_train(training_data, 'male')

# Female DTM

female_dtm = dtm_train(training_data, 'female')

## Train classifiers


### Functions for various ML models

In [16]:
# Lasso

def fit_logistic_regression(X_train, y_train):
    model = LogisticRegression(penalty = 'l1', # Lasso 
                               solver = 'liblinear') # for small datasets
    # sage solver is faster but doesn't coverge in this case
    model.fit(X_train, y_train)
    return model

# Naive-Bayes 

def fit_bayes(X_train, y_train):
    model = GaussianNB()
    model.fit(X_train, y_train)
    return model

# Xgboost

def fit_xgboost(X_train, y_train):
    model = XGBClassifier(random_state = 42,
                         seed = 2, 
                         colsample_bytree = 0.6,
                         subsample = 0.7)
    model.fit(X_train, y_train)
    return model

### Function for evaluating ML models (accuracy and balanced accuracy)

In [22]:
def test_model(model, X_train, y_train, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return(accuracy, precision, recall)

### Model fitting 

In [18]:
def fit_models(data):
    # Lasso
    lasso = fit_logistic_regression(data[0], data[1])
    # Naive-Bayes
    bayes = fit_bayes(data[0], data[1])
    # Xgboost
    xgboost = fit_xgboost(data[0], data[1])
    
    return(lasso, bayes, xgboost)

In [19]:

male_fit = fit_models(male_dtm)

female_fit = fit_models(female_dtm)


## Model evaluations 

### Function for testing multiple models

In [23]:
def test_models(models, data):
    lasso = test_model(models[0], data[0], data[1], data[2], data[3]) 
    bayes = test_model(models[1], data[0], data[1], data[2], data[3])
    xgboost = test_model(models[2], data[0], data[1], data[2], data[3])
    return(lasso, bayes, xgboost)

Evaluate multiple models for each data.

In [43]:
male_models = test_models(male_fit, male_dtm)

female_models = test_models(female_fit, female_dtm)

### Function for putting the model evaluations into a table.

In [44]:
# Define evaluation function 

def eval_table(data):
    # Turn it into a data frame 
    table = pd.DataFrame(list(data), columns= ['Accuracy','Precision','Recall']) 
    # Add column names 
    table.insert(loc = 0, column = 'Models', value = ['Lasso', 'Bayes', 'XGBoost'])
    # Round to 2 decimals 
    table = round(table, 2)
    return(table)

In [47]:
# Apply to each model outcomes and add gender columns 

eval_male_models = eval_table(male_models)
eval_male_models['Label'] = 'Male'

eval_feamle_models = eval_table(female_models)
eval_feamle_models['Label'] = 'Female'

In [51]:
# Merge the two data frames 

eval_gender_models = pd.concat([eval_male_models, eval_feamle_models])

eval_gender_models 


,Models,Accuracy,Precision,Recall,Label
0,Lasso,0.69,0.68,0.73,Male
1,Bayes,0.62,0.58,0.86,Male
2,XGBoost,0.65,0.65,0.65,Male
0,Lasso,0.73,0.73,0.77,Female
1,Bayes,0.69,0.65,0.84,Female
2,XGBoost,0.71,0.72,0.71,Female


In [53]:
# Export the data 

eval_gender_models.to_csv("/home/jae/intersectional-bias-in-ml/processed_data/eval_gender_models.csv")

## Prediction

### Function for predicting the unlabeled data (tweets)

In [ ]:
def predict_text(text, model):   
      
    # BOW model 
    
    features = vectorizer.fit_transform(text).todense()
    
    # Prediction
    
    preds = model.predict(features)
    
    return preds

### Apply the function to the tweets

In [ ]:
male_predicted = predict_text(tweets_clean['text'], male_fit[0])

In [ ]:
female_predicted = predict_text(tweets_clean['text'], female_fit[0])

In [ ]:
fin_results = []

for i in np.arange(len(male_predicted)):
    if male_predicted[i] == 1 and female_predicted[i] == 0:
        fin_results.append('male')
    elif male_predicted[i] == 0 and female_predicted[i] == 1:
        fin_results.append('female')
    elif male_predicted[i] == 1 and female_predicted[i] == 1:
        fin_results.append('both')
    else:
        fin_results.append('neither')

        
tweets_clean['gender'] = fin_results

### Data quality check 

In [ ]:
# The only data cleaning that was done was to lower case everything.
# Verifying there are no null entries in the tweet text.
tweets_clean['text'].isnull().values.any()

In [ ]:
tweets_clean['gender'].value_counts()

In [ ]:
tweets_clean['male'] = male_predicted
tweets_clean['female'] = female_predicted

In [ ]:
tweets_clean['male'].value_counts()

In [ ]:
tweets_clean['female'].value_counts()

In [ ]:
tweets_clean.head()

## Export the predicted values 

In [ ]:
# tweets_clean.columns

In [ ]:
# tweets_clean.to_csv("/home/jae/intersectional-bias-in-ml/processed_data/gender_predictions.csv", sep=',', encoding='utf-8', 
          #          header=["text", "label", "votes", "gender", "male", "female"], index=True)